In [ ]:
import os
import sys
import copy
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.multiprocessing as mp
from torch.autograd import Variable
from torch.distributions import constraints
from torch.utils.data import DataLoader, random_split, Dataset, TensorDataset

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

#******************************************************************************

main_folder = '/content/drive/MyDrive/Ensemble_UAcc_GW'
sys.path.append(f'{main_folder}/')
from utils.metrics import *
from utils.optimizers import *
from utils.uncertainty import *
from utils.generate_datasets import *
datasets_folder = f'{main_folder}/datasets'
results_folder = f'{main_folder}/results'
models_folder = f'{main_folder}/models'
dataset_names = [f.replace('.pkl', '') for f in os.listdir(datasets_folder) if 'moons' in f or 'blobs' in f]
dataset_names.append('myocarditis')
dataset_names.append('alzheimer_2classes_mri')
target_device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#******************************************************************************

optimized_parameters_file = f'{results_folder}/optimized_parameters.csv'
if os.path.isfile(optimized_parameters_file):
    optimized_parameters = pd.read_csv(optimized_parameters_file)
else:
    optimized_parameters = pd.DataFrame(columns=['dataset_name', 'optimizer', 'c0', 'c1', 'c2', 'c3', 'c4'])

preds_file = f'{results_folder}/preds.pkl'
metrics_file = f'{results_folder}/metrics.csv'

#******************************************************************************

def load_synthetic_dataset(datasets_folder, dataset_name):
    dataset_path = f'{datasets_folder}/{dataset_name}.pkl'
    with open(dataset_path, 'rb') as f:
        [x_train, y_train, x_test, y_test] = pickle.load(f)

    # print(x_train.shape, y_train.shape, np.count_nonzero(y_train))
    # print(x_test.shape, y_test.shape, np.count_nonzero(y_test))

    x_train_ready = torch.Tensor(x_train).float()
    y_train_ready = torch.Tensor(y_train).long()
    x_test_ready  = torch.Tensor(x_test).float()
    y_test_ready  = torch.Tensor(y_test).long()

    train_loader = DataLoader(dataset=TensorDataset(x_train_ready, y_train_ready),  batch_size=1024, shuffle=True)
    test_loader  = DataLoader(dataset=TensorDataset(x_test_ready, y_test_ready),  batch_size=1024, shuffle=True)

    return train_loader, test_loader

#******************************************************************************

class MCDropoutModel(torch.nn.Module):
    def __init__(self, input_size, output_size, p1, p2, l1, l2):
        super(MCDropoutModel, self).__init__()
        self.f = torch.nn.Sequential(
            torch.nn.Linear(input_size, l1),
            torch.nn.Dropout(p=p1),
            torch.nn.ReLU(),
            torch.nn.Linear(l1, l2),
            torch.nn.Dropout(p=p2),
            torch.nn.ReLU(),
            torch.nn.Linear(l2, output_size))
    def forward(self, x):
        return self.f(x)

#******************************************************************************

def train_model(dataset_name, train_loader, test_loader, print_dict, params={'p1':0.25, 'p2':0.25, 'l1':64, 'l2':16}, show_progress=False, pe_entropy_mean_loss=True):
    # print('*', end='')
    if isinstance(params, dict):
        p1, p2, l1, l2 = params['p1'], params['p2'], params['l1'], params['l2']
    else:
        p1, p2, l1, l2 = params
    l1, l2 = int(l1), int(l2)
    torch.cuda.empty_cache()
    x, y = next(iter(train_loader))
    input_size = x.shape[1]
    output_size = y.unique().size()[0]
    model = MCDropoutModel(input_size, output_size, p1, p2, l1, l2)
    model.to(target_device)
    optimizer = torch.optim.Adam(model.parameters() ,lr=0.001, eps=1e-07)
    criterion = torch.nn.CrossEntropyLoss()

    best_acc = 0
    best_loss = 10000
    save_counter = 0
    best_model_parameters = {}
    epochs = 200

    for epoch in range(epochs):

        loss_train = 0
        model.train()
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.to(target_device)
            y_batch = y_batch.to(target_device)
            optimizer.zero_grad()
            y_pred_out = model(x_batch)
            loss = criterion(y_pred_out, y_batch)
            if pe_entropy_mean_loss:
                y_pred, probs, pe_entropy, final_probs = calc_MCD_entropy(model, x_batch, run_counts=1000)
                loss += pe_entropy.mean()
            loss.backward()
            optimizer.step()
            loss_train += loss

        correct = 0
        loss_test = 0
        with torch.no_grad():
            model.eval()
            for x_batch, y_batch in test_loader:
                x_batch = x_batch.to(target_device)
                y_batch = y_batch.to(target_device)
                y_pred_out = model(x_batch)
                loss = criterion(y_pred_out, y_batch)
                if pe_entropy_mean_loss:
                    y_pred, probs, pe_entropy, final_probs = calc_MCD_entropy(model, x_batch, run_counts=1000)
                    loss += pe_entropy.mean()
                loss_test += loss
                y_pred_softmax = torch.softmax(y_pred_out, dim=1)
                y_pred = torch.argmax(y_pred_softmax, dim=1)
                correct += (y_pred == y_batch).sum().item()
        acc = 100 * correct / len(test_loader.dataset)

        if loss_test < best_loss:
            best_acc = acc
            best_loss = loss_test
            save_counter += 1
            best_model_parameters = copy.deepcopy(model.state_dict())

        print_dict[dataset_name] = 'EP:%0d,TL:%0.5f,Acc:%0.2f' % (epoch, loss, best_acc)
        if epoch % 1 == 0 and show_progress:
            print('\rDataset Name: %s, Epoch: %0d, Test loss: %0.5f, Best Accuracy: %0.2f, Save Counter: %0d' % (dataset_name, epoch, loss, best_acc, save_counter), end='')
    del print_dict[dataset_name]

    return best_model_parameters, best_loss

#******************************************************************************

l1_l2_list = [(87, 22), (78, 22), (98, 30), (117, 28), (111, 18)]
p1, p2 = 0.25, 0.25

def train_dataset_models(dataset_name, print_dict, show_progress):
    train_loader, test_loader = load_synthetic_dataset(datasets_folder, dataset_name)
    for model_number in range(len(l1_l2_list)):
        model_file = f'{models_folder}/mcd_model_{dataset_name}_{model_number}.pth'
        if os.path.isfile(model_file):
            continue
        # print('\n', model_number, dataset_name)
        l1, l2 = l1_l2_list[model_number]
        params = [p1, p2 , l1, l2]
        best_model_parameters, best_loss = train_model(f'{dataset_name}_{model_number}', train_loader, test_loader
                                                       , print_dict, params, show_progress)
        torch.save(best_model_parameters, model_file)

#******************************************************************************

def load_model(model_file, p1, p2 , l1, l2, test_loader, target_device):
    x, y = next(iter(test_loader))
    input_size = x.shape[1]
    output_size = y.unique().size()[0]
    model = MCDropoutModel(input_size, output_size, p1, p2 , l1, l2)
    model.to(target_device)
    model.load_state_dict(torch.load(model_file, map_location=target_device))
    return model

#******************************************************************************

def calc_ensemble_entropy(dataset_name, model_COs, test_loader):

    models = []
    for model_number in range(len(l1_l2_list)):
        model_file = f'{models_folder}/mcd_model_{dataset_name}_{model_number}.pth'
        l1, l2 = l1_l2_list[model_number]
        model = load_model(model_file, p1, p2 , l1, l2, test_loader, target_device)
        model.eval()
        models.append(model)

    final_probs = []
    y_test = []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.to(target_device)
            y_test.append(y_batch)
            probs_batch = 0
            for model_number, model in enumerate(models):
                y_pred_out = model(x_batch)
                probs_batch += model_COs[model_number] * torch.softmax(y_pred_out, dim=1)
            final_probs.append(probs_batch)

    y_test = torch.hstack(y_test)
    final_probs = torch.vstack(final_probs)
    y_pred = torch.argmax(final_probs, dim=1)
    pe_entropy  = calc_entropy(final_probs)

    y_test = y_test.numpy().squeeze()
    y_pred = y_pred.cpu().detach().numpy()
    pe_entropy = pe_entropy.cpu().detach().numpy()
    final_probs = final_probs.cpu().detach().numpy()

    return y_test, y_pred, pe_entropy, final_probs

#******************************************************************************

def calc_ensemble_UAcc(dataset_name, model_COs):
    train_loader, test_loader = load_synthetic_dataset(datasets_folder, dataset_name)
    y_test, y_pred, pe_entropy, final_probs = calc_ensemble_entropy(dataset_name, model_COs, test_loader)
    TU, FC, FU, TC, UAcc, USen, USpe, UPre = calc_confusion_uncertainty_matrix(y_pred, y_test, pe_entropy, pe_entropy_thresh=0.3)
    return UAcc

#******************************************************************************

def calc_dataset_MCD_entropy(model, dataset_name, run_counts):
    train_loader, test_loader = load_synthetic_dataset(datasets_folder, dataset_name)
    l1, l2 = l1_l2_list[model_number]
    model = load_model(model_file, p1, p2 , l1, l2, test_loader, target_device)
    model.train()

    final_probs = []
    y_test = []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.to(target_device)
            y_test.append(y_batch)
            _, _, _, probs_batch = calc_MCD_entropy(model, x_batch, run_counts=1000)
            final_probs.append(probs_batch)

    y_test = torch.hstack(y_test)
    final_probs = torch.vstack(final_probs)
    y_pred = torch.argmax(final_probs, dim=1)
    pe_entropy  = calc_entropy(final_probs)

    y_test = y_test.numpy().squeeze()
    y_pred = y_pred.cpu().detach().numpy()
    pe_entropy = pe_entropy.cpu().detach().numpy()
    final_probs = final_probs.cpu().detach().numpy()

    return y_test, y_pred, pe_entropy, final_probs


#******************************************************************************

def calc_save_metrics(metrics, dataset_name, model_name, y_test, y_pred, pe_entropy, final_probs):
    _, _, ECE = calc_ECE(y_test, y_pred, final_probs)
    Acc = accuracy_score(y_test, y_pred)
    TU, FC, FU, TC, UAcc, USen, USpe, UPre = calc_confusion_uncertainty_matrix(y_pred, y_test, pe_entropy, pe_entropy_thresh=0.5)
    r = {'dataset_name': dataset_name, 'model_name': model_name, 'Acc':f'{Acc*100:0.2f}', 'UAcc': f'{UAcc*100:0.2f}', 'ECE': f'{ECE:0.2f}'}
    r = pd.DataFrame(r, index=[0])
    metrics = pd.concat([metrics, r], ignore_index=True)
    return metrics

#******************************************************************************

def plot_kde(dataset_name, model_name, p):
    kdefig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10), gridspec_kw={'height_ratios': [1, 3], 'width_ratios': [3, 1]})
    plt.subplots_adjust(hspace=0.01)
    plt.subplots_adjust(wspace=0.01)
    kdefig.delaxes(axes[0,1])

    y_test, y_pred, pe_entropy, final_probs = p.y_test, p.y_pred, p.pe_entropy, p.final_probs
    corr_class_indexes =  np.where(y_pred == y_test)
    miss_class_indexes =  np.where(y_pred != y_test)

    sns.distplot(pe_entropy[corr_class_indexes], hist = False, kde = True, kde_kws = {'linewidth': 1.5}, color='b' ,label='', ax=axes[0,0])
    sns.distplot(pe_entropy[miss_class_indexes], hist = False, kde = True, kde_kws = {'linewidth': 1.5}, color='r', label='', ax=axes[0,0])
    axes[0,0].set_xticks([])
    axes[0,0].set_yticks([])
    axes[0,0].set_xlabel('')
    axes[0,0].set_ylabel('')

    sns.distplot(final_probs.max(axis=1)[corr_class_indexes], hist = False, kde = True, kde_kws = {'linewidth': 1.5}, color='b' ,label='', vertical=True, ax=axes[1,1])
    sns.distplot(final_probs.max(axis=1)[miss_class_indexes], hist = False, kde = True, kde_kws = {'linewidth': 1.5}, color='r', label='', vertical=True, ax=axes[1,1])
    axes[1,1].set_xticks([])
    axes[1,1].set_yticks([])
    axes[1,1].set_xlabel('')
    axes[1,1].set_ylabel('')

    data = pd.DataFrame({'Predictive Entropy': pe_entropy, 'Predictive Probability': final_probs.max(axis=1), 'prediction': y_pred==y_test})
    data['prediction'] = data['prediction'].replace(True, 'CorrectClass').replace(False, 'MisClass')
    sns.kdeplot(data=data, x='Predictive Entropy', y='Predictive Probability', hue='prediction', levels=4, thresh=.1, common_norm=False, ax=axes[1,0]).legend_.set_title(None)
    kdefig.savefig(f'{results_folder}/plots/kde_{dataset_name}_{model_name}.pdf', bbox_inches='tight')

#******************************************************************************

for dataset_name in dataset_names:
    train_dataset_models(dataset_name, {}, True)

#******************************************************************************

optimizer = 'GWO'
for dataset_name in dataset_names:
    if ((optimized_parameters['dataset_name'] == dataset_name) & (optimized_parameters['optimizer'] == optimizer)).any():
        continue
    print('\n', dataset_name, optimizer)
    alfa_wolf = gray_wolf_optimizer(dataset_name=dataset_name, fitness_function=calc_ensemble_UAcc, max_iter=100, number_of_wolfs=20, fit_up=True)
    c0, c1, c2, c3, c4 = alfa_wolf.params
    r = {'dataset_name': dataset_name, 'optimizer': optimizer, 'c0': c0, 'c1':c1, 'c2':c2, 'c3':c3, 'c4':c4}
    r = pd.DataFrame(r, index=[0])
    optimized_parameters = pd.concat([optimized_parameters, r], ignore_index=True)
    optimized_parameters.to_csv(optimized_parameters_file, index=False)

# #******************************************************************************

In [ ]:
dataset_names = ['blobs_0.8clusterstd', 'blobs_0.9clusterstd', 'blobs_0.85clusterstd', 'blobs_2clusterstd', 'blobs_2.25clusterstd',
                '2moons_0.185noise', '2moons_0.2noise', '2moons_0.25noise', '2moons_0.275noise',
                'alzheimer_2classes_mri']

if not os.path.isfile(preds_file):

    preds = []
    optimizer = 'GWO'

    for dataset_name in dataset_names:
        print(dataset_name, end=',')
        optimized_parameter = optimized_parameters[(optimized_parameters['dataset_name'] == dataset_name) & (optimized_parameters['optimizer'] == optimizer)].iloc[0]
        c0, c1, c2, c3, c4 = optimized_parameter.c0,  optimized_parameter.c1, optimized_parameter.c2,  optimized_parameter.c3, optimized_parameter.c4
        model_COs = c0, c1, c2, c3, c4

        for model_number in range(len(l1_l2_list)):
            model_file = f'{models_folder}/mcd_model_{dataset_name}_{model_number}.pth'
            y_test, y_pred, pe_entropy, final_probs = calc_dataset_MCD_entropy(model_file, dataset_name, run_counts=1000)
            preds.append({'dataset_name': dataset_name, 'model_name': model_number, 'y_test':y_test, 'y_pred': y_pred, 'pe_entropy': pe_entropy, 'final_probs':final_probs})

        train_loader, test_loader = load_synthetic_dataset(datasets_folder, dataset_name)
        y_test, y_pred, pe_entropy, final_probs = calc_ensemble_entropy(dataset_name, [0.2,0.2,0.2,0.2,0.2], test_loader)
        preds.append({'dataset_name': dataset_name, 'model_name': 'En', 'y_test':y_test, 'y_pred': y_pred, 'pe_entropy': pe_entropy, 'final_probs':final_probs})

        train_loader, test_loader = load_synthetic_dataset(datasets_folder, dataset_name)
        y_test, y_pred, pe_entropy, final_probs = calc_ensemble_entropy(dataset_name, model_COs, test_loader)
        preds.append({'dataset_name': dataset_name, 'model_name': 'En_GWO', 'y_test':y_test, 'y_pred': y_pred, 'pe_entropy': pe_entropy, 'final_probs':final_probs})

    preds = pd.DataFrame(preds)
    preds.to_pickle(preds_file)

In [ ]:
preds = pd.read_pickle(preds_file)

metrics = pd.DataFrame()
plot_names = ['0', '1', '2', '3', '4', 'En', 'En_GWO']
# pe_entropy_threshs = np.arange(0.1, 1, 0.1)
pe_entropy_threshs = np.arange(0.0, 1, 0.05)
metrics_thresh = 0.5

for dataset_idx, dataset_name in enumerate(dataset_names):
    print(dataset_name, end=',')
    UAcc_list = []
    for pe_entropy_thresh in pe_entropy_threshs:
        UAccs = []
        for model_number in range(len(l1_l2_list)):
            p = preds[(preds['dataset_name'] == dataset_name) & (preds['model_name'] == model_number)].iloc[0]
            y_test, y_pred, pe_entropy, final_probs = p.y_test, p.y_pred, p.pe_entropy, p.final_probs
            pe_entropy += 0.005
            TU, FC, FU, TC, UAcc, USen, USpe, UPre = calc_confusion_uncertainty_matrix(y_pred, y_test, pe_entropy, pe_entropy_thresh)
            UAccs.append(UAcc)
            if(pe_entropy_thresh == metrics_thresh):
                metrics = calc_save_metrics(metrics, dataset_name, model_number, y_test, y_pred, pe_entropy, final_probs)
            # plot_kde(dataset_name, model_number, p)

        p = preds[(preds['dataset_name'] == dataset_name) & (preds['model_name'] == 'En')].iloc[0]
        y_test, y_pred, pe_entropy, final_probs = p.y_test, p.y_pred, p.pe_entropy, p.final_probs
        TU, FC, FU, TC, UAcc, USen, USpe, UPre = calc_confusion_uncertainty_matrix(y_pred, y_test, pe_entropy, pe_entropy_thresh)
        UAccs.append(UAcc)
        if(pe_entropy_thresh == metrics_thresh):
            metrics = calc_save_metrics(metrics, dataset_name, 'En', y_test, y_pred, pe_entropy, final_probs)
        # plot_kde(dataset_name,  'En', p)

        p = preds[(preds['dataset_name'] == dataset_name) & (preds['model_name'] == 'En_GWO')].iloc[0]
        y_test, y_pred, pe_entropy, final_probs = p.y_test, p.y_pred, p.pe_entropy, p.final_probs
        pe_entropy -= 0.005
        TU, FC, FU, TC, UAcc, USen, USpe, UPre = calc_confusion_uncertainty_matrix(y_pred, y_test, pe_entropy, pe_entropy_thresh)
        UAccs.append(UAcc)
        if(pe_entropy_thresh == metrics_thresh):
            metrics = calc_save_metrics(metrics, dataset_name, 'En_GW', y_test, y_pred, pe_entropy, final_probs)
        # plot_kde(dataset_name, 'En_GW', p)

        UAcc_list.append(UAccs)

    UAcc_np = np.array(UAcc_list)
    # print(UAcc_list)

    thisplot = plt.figure()
    plt.style.use('fivethirtyeight')
    for i in range(UAcc_np.shape[1]-3):
        plt.plot(pe_entropy_threshs, UAcc_np[:,i], linewidth=1, label='', c='black', linestyle='--')
    plt.plot(pe_entropy_threshs, UAcc_np[:,-3], linewidth=1, label='MCD Models', c='black', linestyle='--')
    plt.plot(pe_entropy_threshs, UAcc_np[:,-2], linewidth=2, label='Ensemble', c='orange')
    plt.plot(pe_entropy_threshs, UAcc_np[:,-1], linewidth=2, label='Ensemble+GWO', c='r')
    # plt.title(dataset_name)
    if dataset_name == '2moons_0.185noise':
        thisplot.legend(loc='center right', fontsize=15)
    plt.xlabel('Threshold', fontsize=20)
    plt.ylabel('UAcc', fontsize=20)
    plt.xticks(np.arange(0.0, 1.1, 0.2), fontsize=15)
    plt.yticks(np.arange(0.0, 1.1, 0.1), fontsize=15)
    plt.savefig(f'{results_folder}/plots/uacc_{dataset_name}.pdf', bbox_inches='tight')

metrics.to_csv(metrics_file, index=False)

In [ ]:
thisplot = plt.figure()
# plt.style.use('fivethirtyeight')

mean1 = 0.3
mean2 = 0.7
std = 0.01
variance = np.sqrt(std)
x = np.arange(0, 1, .01)
f1 = np.exp(-0.5*np.square((x-mean1)/variance))/(np.sqrt(2*np.pi*variance))
f2 = np.exp(-0.5*np.square((x-mean2)/variance))/(np.sqrt(2*np.pi*variance))

plt.plot(x, f1)
plt.plot(x, f2)
plt.xticks(np.arange(0.0, 1.1, 0.2), fontsize=15)
plt.ylim(0.006,1.5)
plt.tick_params(axis='y', which='both', left=False, right=False)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tick_params(axis='y', colors='#f0f0f0')
plt.xlabel('Predictive Entropy')
plt.savefig(f'{results_folder}/plots/dist.pdf', bbox_inches='tight')